In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from nltk.corpus import stopwords
import re

In [3]:
train = pd.read_csv("labeledTrainData.tsv", header=0,\
                    delimiter="\t", quoting=3)

test = pd.read_csv("testData.tsv",header=0,\
                    delimiter="\t", quoting=3)

In [4]:
def review_word_list(review):
    review_text=BeautifulSoup(review,'lxml').get_text()
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    words= review_text.lower().split()
    words= [w for w in words if not w in set(stopwords.words('english'))]
    return words


In [5]:
import nltk.data
tokenizer= nltk.data.load('tokenizers/punkt/english.pickle')

In [9]:
def review_sentence(review, tokenizer):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences=[]
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            a=review_word_list(raw_sentence)
            sentences.append(a)
    
    return sentences

In [12]:
sentences=[]
i=0
for review in train['review']:
    if i%1000==0:
        print (i , "out of ", len(train['review']))
    
    sentences+=review_sentence(review,tokenizer)
    i=i+1

0 out of  25000


C:\Users\tomar\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


1000 out of  25000


C:\Users\tomar\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\tomar\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


2000 out of  25000
3000 out of  25000
4000 out of  25000
5000 out of  25000
6000 out of  25000
7000 out of  25000
8000 out of  25000
9000 out of  25000
10000 out of  25000
11000 out of  25000
12000 out of  25000
13000 out of  25000
14000 out of  25000
15000 out of  25000
16000 out of  25000
17000 out of  25000
18000 out of  25000
19000 out of  25000
20000 out of  25000
21000 out of  25000
22000 out of  25000
23000 out of  25000
24000 out of  25000


In [13]:
num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

C:\Users\tomar\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Training model....


In [117]:
model_name = "300features_40minwords_10context"
model.save(model_name)

In [14]:
model.wv.doesnt_match("man woman dog child planet".split())

C:\Users\tomar\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'planet'

In [15]:
model.wv.doesnt_match("france england germany berlin".split())

C:\Users\tomar\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'berlin'

In [16]:
model.wv.most_similar("woman")

C:\Users\tomar\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lady', 0.694207489490509),
 ('sexually', 0.691582441329956),
 ('lover', 0.6900498270988464),
 ('girl', 0.6889432668685913),
 ('loretta', 0.6845369338989258),
 ('giovanna', 0.6834275722503662),
 ('husband', 0.6786994934082031),
 ('man', 0.6738367676734924),
 ('boy', 0.6618785262107849),
 ('brides', 0.6567746996879578)]

In [17]:
model.wv.most_similar("awful")

C:\Users\tomar\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.8852403163909912),
 ('horrible', 0.863947868347168),
 ('atrocious', 0.7818443775177002),
 ('dreadful', 0.7764734029769897),
 ('crappy', 0.768346905708313),
 ('abysmal', 0.7656266689300537),
 ('sucks', 0.7594637870788574),
 ('pathetic', 0.7486364841461182),
 ('lousy', 0.7310115098953247),
 ('horrid', 0.7291818857192993)]

In [19]:
def features(words,num_features,model):
    feature= np.zeros(num_features,dtype='float32')
    nwords=0
    
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords=nwords+1
            
            feature=np.add(feature,model[word])
    
    feature=np.divide(feature,nwords)
    return feature

In [20]:
def avg_features(reviews,num_features,model):
    counter=0
    avg_feature_vec=np.zeros((len(reviews),num_features),dtype='float32')
    for review in reviews:
        avg_feature_vec[counter]=features(review,num_features,model)
        counter=counter+1
    
    return avg_feature_vec

In [23]:
clean_train_reviews = []
i=0
for review in train['review']:
    if i%1000==0:
        print (i , "out of ", len(train['review']))
        
    clean_train_reviews.append(review_word_list(review))
    i=i+1

train_data_vector = avg_features(clean_train_reviews,num_features,model)

0 out of  25000
1000 out of  25000
2000 out of  25000
3000 out of  25000
4000 out of  25000
5000 out of  25000
6000 out of  25000
7000 out of  25000
8000 out of  25000
9000 out of  25000
10000 out of  25000
11000 out of  25000
12000 out of  25000
13000 out of  25000
14000 out of  25000
15000 out of  25000
16000 out of  25000
17000 out of  25000
18000 out of  25000
19000 out of  25000
20000 out of  25000
21000 out of  25000
22000 out of  25000
23000 out of  25000
24000 out of  25000


C:\Users\tomar\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [24]:
clean_test_reviews = []
i=0
for review in test['review']:
    if i%1000==0:
        print (i , "out of ", len(test['review']))
        
    clean_test_reviews.append(review_word_list(review))
    i=i+1
test_data_vector = avg_features(clean_test_reviews,num_features,model)

0 out of  25000
1000 out of  25000
2000 out of  25000
3000 out of  25000
4000 out of  25000
5000 out of  25000
6000 out of  25000
7000 out of  25000
8000 out of  25000
9000 out of  25000
10000 out of  25000
11000 out of  25000
12000 out of  25000
13000 out of  25000
14000 out of  25000
15000 out of  25000
16000 out of  25000
17000 out of  25000
18000 out of  25000
19000 out of  25000
20000 out of  25000
21000 out of  25000
22000 out of  25000
23000 out of  25000
24000 out of  25000


C:\Users\tomar\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [30]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()   
print("Fitting Logistic Regression to training data....")    
model.fit(train_data_vector, train["sentiment"])
pred=model.predict(train_data_vector)
model.score(train_data_vector,train["sentiment"])

Fitting Logistic Regression to training data....


0.85144

In [31]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(train["sentiment"],pred))

[[10529  1971]
 [ 1743 10757]]


In [32]:
from sklearn.metrics import classification_report
print(classification_report(train["sentiment"],pred))

             precision    recall  f1-score   support

          0       0.86      0.84      0.85     12500
          1       0.85      0.86      0.85     12500

avg / total       0.85      0.85      0.85     25000

